# I più semplici esempi di interazione con modelli locali via l'API di OpenAI

Luca Mari, maggio 2025  

Quest'opera è distribuita con <a href="http://creativecommons.org/licenses/by-nc-sa/4.0" target="_blank">Licenza Creative Commons Attribuzione - Non commerciale - Condividi allo stesso modo 4.0 Internazionale</a>.  
<img src="https://creativecommons.it/chapterIT/wp-content/uploads/2021/01/by-nc-sa.eu_.png" width="100">

**Obiettivo**: comprendere le più semplice modalità di interazione con modelli locali via l'API di OpenAI.  
**Precompetenze**: basi di Python.

> Per eseguire questo notebook con VSCode sul proprio calcolatore, occorre:
> * installare un interprete Python
> * attivare un server locale che renda possibile l'interazione con un modello via l'API di OpenAI (per semplicità, supporremo che sia LM Studio, scaricabile da https://lmstudio.ai)
> * scaricare da https://code.visualstudio.com/download e installare VSCode
> * eseguire VSCode e attivare le estensioni per Python e Jupyter
> * ancora in VSCode:
>     * creare una cartella di lavoro e renderla la cartella corrente
>     * copiare nella cartella il file di questa attività: [oaibase.ipynb](oaibase.ipynb)
>     * aprire il notebook `oaibase.ipynb`
>     * creare un ambiente virtuale locale Python (Select Kernel | Python Environments | Create Python Environment | Venv, e scegliere un interprete Python):
>     * installare i moduli Python richiesti, eseguendo dal terminale:  
>         `pip install openai`

Importiamo i moduli Python necessari, definiamo alcune funzioni di utilità, e specifichiamo l'_end point_ per l'accesso al server locale.

In [6]:
from openai import OpenAI
from pprint import pprint
from IPython.display import Markdown, display
import json

def stream_print(response, max_length=100):
    length = 0
    for chunk in response:
        text = chunk.choices[0].delta
        if hasattr(text, 'content') and text.content:
            print(text.content, end='', flush=True)
            length += len(text.content)
            if length > max_length:
                print()
                length = 0

def print_markdown(response):
    display(Markdown(response))

def stream_print_markdown(response):
    buffer = ""
    for chunk in response:
        text = chunk.choices[0].delta
        if hasattr(text, 'content') and text.content:
            buffer += text.content
            display(Markdown(buffer), clear=True)


client = OpenAI(base_url="http://localhost:1234/v1", api_key="...") # usa un server locale, per esempio con LM Studio

### Elenco dei modelli disponibili
Assumendo che il server sia attivo, questo è il più semplice esempio di una richiesta al server via l'API di OpenAI, per ottenere la lista dei modelli accessibili attraverso il server stesso (ma in effetti dunque anche per accertare che il server sia accessibile).

In [7]:
models = client.models.list()
pprint(models.data)

[Model(id='gemma-3-4b-it', created=None, object='model', owned_by='organization_owner'),
 Model(id='qwen3-30b-a3b', created=None, object='model', owned_by='organization_owner'),
 Model(id='qwen3-4b', created=None, object='model', owned_by='organization_owner'),
 Model(id='text-embedding-nomic-embed-text-v1.5@q4_k_m', created=None, object='model', owned_by='organization_owner'),
 Model(id='gemma-3-12b-it', created=None, object='model', owned_by='organization_owner'),
 Model(id='qwen2.5-7b-instruct-1m', created=None, object='model', owned_by='organization_owner'),
 Model(id='ministral-4b-instruct', created=None, object='model', owned_by='organization_owner'),
 Model(id='gemma-3-27b-it', created=None, object='model', owned_by='organization_owner'),
 Model(id='qwen2.5-32b-instruct', created=None, object='model', owned_by='organization_owner'),
 Model(id='qwq-32b', created=None, object='model', owned_by='organization_owner'),
 Model(id='simplescaling_s1.1-32b', created=None, object='model',

### Generazione di sentence embedding
Assumendo che sia stato caricato un modello di _embedding_ sul server (per esempio `text-embedding-nomic-embed-text-v1.5@f32`), possiamo usarlo per fare _sentence embedding_. 

In [8]:
text = "La bellezza salverà il mondo."

response = client.embeddings.create(
    model="...",
    input=[text]
).data[0].embedding

print(f'''La frase è stata codificata in un vettore di {len(response)} numeri;
i primi 5 sono: {response[:5]}''')

La frase è stata codificata in un vettore di 768 numeri;
i primi 5 sono: [0.10395540297031403, 0.050489071756601334, -0.0963636264204979, 0.011676624417304993, -0.04801320657134056]


### Completion
Questo è il più semplice esempio di una richiesta al server via l'API di OpenAI (stiamo supponendo che il server sia attivo e sia stato caricato un modello, per esempio `gemma-3-4b-it`).

In [9]:
prompt = "Presentati: chi sei?"

response = client.chat.completions.create(
    model="...",
    messages=[
        {"role": "system", "content": "Rispondi sempre in italiano. /no_think"},
        {"role": "user", "content": prompt}
    ],
    max_tokens=-1,
    temperature=2,
    stream=False
)

print("L'intero oggetto JSON di risposta:\n")
pprint(dict(response))

print("\nIl messaggio generato:")
print_markdown(response.choices[0].message.content)

L'intero oggetto JSON di risposta:

{'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sono Gemma, un modello linguistico di grandi dimensioni creato dal team di Google DeepMind. Sono un'intelligenza artificiale addestrata per comunicare e generare testo simile a quello umano. Sono un modello open-weight, il che significa che sono ampiamente disponibile al pubblico.\n\nCome posso aiutarti oggi?\n", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))],
 'created': 1746359909,
 'id': 'chatcmpl-43758hv5xunu6lvb5e9nyl',
 'model': 'gemma-3-4b-it',
 'object': 'chat.completion',
 'service_tier': None,
 'stats': {},
 'system_fingerprint': 'gemma-3-4b-it',
 'usage': CompletionUsage(completion_tokens=67, prompt_tokens=27, total_tokens=94, completion_tokens_details=None, prompt_tokens_details=None)}

Il messaggio generato:


Sono Gemma, un modello linguistico di grandi dimensioni creato dal team di Google DeepMind. Sono un'intelligenza artificiale addestrata per comunicare e generare testo simile a quello umano. Sono un modello open-weight, il che significa che sono ampiamente disponibile al pubblico.

Come posso aiutarti oggi?


### Completion con stream dei token
Quasi identica all'esempio precedente è la richiesta di una risposta che sia inviata un token per volta.

In [12]:
prompt = "Presentati: chi sei? (metti in grassetto le parole chiave)"

response = client.chat.completions.create(
    model="...",
    messages=[
        {"role": "system", "content": "Rispondi sempre in italiano."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=-1,
    temperature=0.7,
    stream=True
)

stream_print_markdown(response)

Ciao! Sono **un modello linguistico di grandi dimensioni**, creato da Google. In pratica, sono un'**intelligenza artificiale** progettata per comprendere e generare testo. Posso rispondere a domande, tradurre lingue, scrivere diversi tipi di contenuti creativi e molto altro ancora. Sono ancora in fase di **sviluppo**, ma imparo continuamente nuove informazioni e migliorando le mie capacità. 

Cosa posso fare per te oggi?

### Completion con interpretazione del contenuto di un'immagine
Se sul server è attivo un modello multimodale appropriato (per esempio `gemma-3-27b-it` -- modelli della serie `gemma-3` con un minor numero di parametri non funzionano...), la richiesta può includere l'URL di un'immagine e si può chiedere al modello di interpretare il contenuto dell'immagine stessa. In tal caso, per prima cosa occorre convertire l'immagine in formato base64.  
Visualizziamo anche l'immagine.

In [19]:
import requests
from PIL import Image
from io import BytesIO
import base64
from IPython.display import Image as IPImage

url = "https://upload.wikimedia.org/wikipedia/commons/a/aa/Fingandslide.jpg"

def get_image_and_convert_to_base64(url):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    buffer = BytesIO()
    image.save(buffer, format="PNG")
    return "data:image/png;base64," + base64.b64encode(buffer.getvalue()).decode()

image_base64 = get_image_and_convert_to_base64(url)

IPImage(url=url)

Si può ora inviare la richiesta al modello.

In [21]:
response = client.chat.completions.create(
    model="...",
    messages=[
        {"role": "system", "content": "Rispondi sempre in italiano."},
        {"role": "user", "content": [
            {"type": "image_url", "image_url": { "url": image_base64 }},
            {"type": "text", "text": "Descrivi cosa vedi"}
        ]}
    ],
    max_tokens=-1,
    temperature=0.7,
    stream=True
)

stream_print_markdown(response)

Certo, ecco una descrizione di ciò che vedo nell'immagine:

L'immagine mostra un primo piano delle mani di qualcuno che suona una chitarra resofonica (spesso chiamata "dobro"). Ecco i dettagli principali:

*   **Chitarra:** La chitarra è argentata e ha una forma particolare, con dei fori circolari e a goccia sul corpo. Si tratta chiaramente di uno strumento risonante, come si evince dal cono metallico visibile all'interno del corpo della chitarra.
*   **Tecnica:** La persona sta suonando la chitarra usando una tecnica chiamata "slide guitar".  Si può vedere un cilindro nero (il "slide") attorno a uno delle dita della mano sinistra, che viene fatto scorrere lungo le corde per ottenere un suono caratteristico e glissato.
*   **Plettro:** La mano destra sta pizzicando le corde con un plettro.
*   **Abbigliamento:** La persona indossa una maglia nera e pantaloni grigi.

In generale, l'immagine suggerisce un musicista che suona uno strumento particolare e con una tecnica specifica, probabilmente in ambito blues o country.

### Completion con structured output
Solo un poco più complessa è una richiesta in cui si specifica lo schema JSON che vogliamo sia utilizzato nella risposta, nuovamente in accordo all'API di OpenAI, nella specifica _structured output_ (https://platform.openai.com/docs/guides/structured-outputs).

In [22]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "presentazione",
        "strict": "true",
        "schema": {
            "type": "object",
            "properties": {
                "nome": {
                    "type": "string",
                    "description": "Il tuo nome"
                },
                "produttore": {
                    "type": "string",
                    "description": "Il nome del tuo produttore",
                    "enum": ["OpenAI", "Google", "Meta", "other"]
                },
                "caratteristiche principali": {
                    "type": "string",
                    "description": "Le tue caratteristiche principali"
                },
                "caratteristiche secondarie": {
                    "type": "string",
                    "description": "Le tue caratteristiche secondarie"
                }
            },
        "required": ["nome", "produttore", "caratteristiche principali", "caratteristiche secondarie"]
        }
    }
}

prompt = "Presentati: chi sei?"

response = client.chat.completions.create(
    model="...",
    messages=[
        {"role": "system", "content": "Rispondi sempre in italiano." },
        {"role": "user", "content": prompt }
    ],
    max_tokens=-1,
    temperature=0.7,
    stream=True,
    response_format=response_format # type: ignore
)

stream_print(response)

{
  "nome": "Gemini",
  "produttore": "Google",
  "caratteristiche principali": "Sono un modello lingu
istico di grandi dimensioni, addestrato da Google. Posso comunicare e generare testo simile a quello umano
 in risposta a una vasta gamma di prompt e domande. Sono ancora in fase di sviluppo, ma imparo cose nuove
 ogni giorno."
    ,
  "caratteristiche secondarie": "Posso tradurre lingue, scrivere diversi tipi di
 contenuti creativi e rispondere alle tue domande in modo informativo. Cerco sempre di fornire risposte
 accurate e complete, ma a volte potrei commettere errori."
}
 	 	 	 	 	 	 	 	 	 	

### Function calling

Ancora una volta in accordo all'API di OpenAI, un caso particolare di _structured output_ è il _function calling_ (https://platform.openai.com/docs/guides/function-calling), in cui lo schema JSON specifica una lista di funzioni, da trattare appunto come strumenti che possono essere usati.  
Data la maggiore complessità di questo caso, sviluppiamo con qualche dettaglio un esempio.

Supponendo di voler ottenere informazioni dal database della World Bank, e di volerlo fare con una chiamata alla sua API (qui qualche informazione al proposito: https://datahelpdesk.worldbank.org/knowledgebase/articles/898581-api-basic-call-structures), la struttura del sistema sarebbe come in questo diagramma:

![schema](oaibase.drawio.svg)

In [23]:
import requests

request = "https://api.worldbank.org/v2/country/it/indicator/SP.URB.TOTL?date=2021&format=json"
response = requests.get(request).json()

print(f"La richiesta: {request}")
print("\nLa risposta:")
pprint(response)

La richiesta: https://api.worldbank.org/v2/country/it/indicator/SP.URB.TOTL?date=2021&format=json

La risposta:
[{'lastupdated': '2025-04-15',
  'page': 1,
  'pages': 1,
  'per_page': 50,
  'sourceid': '2',
  'total': 1},
 [{'country': {'id': 'IT', 'value': 'Italy'},
   'countryiso3code': 'ITA',
   'date': '2021',
   'decimal': 0,
   'indicator': {'id': 'SP.URB.TOTL', 'value': 'Urban population'},
   'obs_status': '',
   'unit': '',
   'value': 42189154}]]


Non è difficile elaborare il JSON della risposta per produrre un risultato più leggibile:

In [24]:
print(f"Paese: {response[1][0]['country']['value']}")
print(f"Popolazione: {response[1][0]['value']}")

Paese: Italy
Popolazione: 42189154


Ma rimane il fatto che la richiesta deve essere inviata rispettando il formato specificato nella API della World Bank.  
Possiamo però usare un modello di linguaggio, a cui porre richieste in italiano, e facendo il modo che queste richieste vengano opportunamente tradotte e quindi eseguite come chiamate all'API, costruendo un sistema più complesso:

![schema2](oaibase2.drawio.svg)

Lo strumento per inviare richieste all'API è in questo caso una semplice funzione Python, insieme con una sua descrizione JSON che consenta al modello di linguaggio di conoscere il nome della funzione e i suoi argomenti. In questo modo, l'oggetto JSON che il modello di linguaggio genera, in risposta a una richiesta che riceve, dovrebbe contenere l'informazione per eseguire la funzione, e quindi inviare all'API la richiesta nel formato opportuno.

In [25]:
def WB_API_call(countries: list[str], date: str) -> list:
    result = []
    for country in countries:
        response = requests.get(f"https://api.worldbank.org/v2/country/{country}/indicator/SP.URB.TOTL?date={date}&format=json")
        result.append(response.json())
    return result

tools = [
    {
        "type": "function",
        "function": {
            "name": "WB_API_call",
            "description": "Ottieni informazioni dal database della World Bank via API",
            "parameters": {
                "type": "object",
                "properties": {
                    "countries": {
                        "type": "list",
                        "description": "Il codice ISO dei Paesi di cui si vogliono ottenere i dati"
                    },
                    "date": {
                        "type": "string",
                        "description": "L'anno di riferimento"
                    }
                },
                "required": ["countries", "date"]
            }
        }
    }
]

Il modello di linguaggio opera perciò come traduttore, da richieste dell'utente formulate in italiano, a JSON per chiamare lo strumento (da cui il termine _function calling_). Vediamone un esempio.

In [26]:
prompt = "Ottieni dal database della World Bank i dati relativi alla popolazione urbana dell'Italia' nel 2021."

response = client.chat.completions.create(
    model="...",
    messages=[
        {"role": "system", "content": "Rispondi sempre in italiano." },
        {"role": "user", "content": prompt }
    ],
    max_tokens=-1,
    temperature=0.7,
    tools=tools # type: ignore
)

pprint(dict(response))

{'choices': [Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='273265957', function=Function(arguments='{"countries":["IT"],"date":"2021"}', name='WB_API_call'), type='function')]))],
 'created': 1746360791,
 'id': 'chatcmpl-tvzg6mspo34ywzfmogvn',
 'model': 'gemma-3-4b-it',
 'object': 'chat.completion',
 'service_tier': None,
 'stats': {},
 'system_fingerprint': 'gemma-3-4b-it',
 'usage': CompletionUsage(completion_tokens=42, prompt_tokens=479, total_tokens=521, completion_tokens_details=None, prompt_tokens_details=None)}


Dato che il formato di questo JSON è standard, la funzione per eseguire lo strumento specificato con gli argomenti specificati è generica:

In [27]:
def exec_tool(response, with_log=False):
    if with_log:
        print("L'intero oggetto JSON di risposta:")
        pprint(dict(response))
        print("\nIl messaggio generato:")
        pprint(response.choices[0].message)

    if response.choices[0].message.tool_calls:
        tool_call = response.choices[0].message.tool_calls[0]

        if with_log:
            print("\nLa parte del messaggio riferita alla funzione da chiamare:")
            print(tool_call.function)

        function_name = tool_call.function.name
        function_arguments = json.loads(tool_call.function.arguments)
        
        if with_log:
            pprint(f"\nLa funzione da chiamare è: {function_name}; i suoi argomenti sono: {function_arguments}")

        result = globals()[function_name](**function_arguments)

        if with_log:
            pprint(f"\nIl risultato della funzione è: {result}")

        return result
    else:
        return None

Ed ecco finalmente un esempio di _function calling_ completo.

In [28]:
prompt = "Ottieni dal database della World Bank i dati relativi alla popolazione urbana delle nazioni del Sud America nel 2019."

response = client.chat.completions.create(
    model="...",
    messages=[
        {"role": "system", "content": "Rispondi sempre in italiano." },
        {"role": "user", "content": prompt }
    ],
    max_tokens=-1,
    temperature=0.7,
    tools=tools # type: ignore
)

final_response = exec_tool(response, with_log=True)
pprint(final_response)

L'intero oggetto JSON di risposta:
{'choices': [Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='792641581', function=Function(arguments='{"countries":["BR","AR","CL","PY","VE","EC","BO"],"date":"2019"}', name='WB_API_call'), type='function')]))],
 'created': 1746360809,
 'id': 'chatcmpl-4jjviafd9uikdx1w243vlg',
 'model': 'gemma-3-4b-it',
 'object': 'chat.completion',
 'service_tier': None,
 'stats': {},
 'system_fingerprint': 'gemma-3-4b-it',
 'usage': CompletionUsage(completion_tokens=60, prompt_tokens=481, total_tokens=541, completion_tokens_details=None, prompt_tokens_details=None)}

Il messaggio generato:
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='792641581', function=Function(argume

E la funzione che estrae i dati rilevanti dalla lista ottenuta. 

In [29]:
def post_process_WB_API_call(response: list) -> list:
    result = []
    for country_response in response:
        country = country_response[1][0]['country']['value']
        population = country_response[1][0]['value']
        result.append({"country": country, "population": population})
    return result

post_process_WB_API_call(final_response) # type: ignore

[{'country': 'Brazil', 'population': 180121128},
 {'country': 'Argentina', 'population': 41371540},
 {'country': 'Chile', 'population': 16825479},
 {'country': 'Paraguay', 'population': 4031453},
 {'country': 'Venezuela, RB', 'population': 25534978},
 {'country': 'Ecuador', 'population': 11095186},
 {'country': 'Bolivia', 'population': 8143476}]

Questo è dunque l'esempio completo.

In [30]:
prompt = "Ottieni dal database della World Bank i dati relativi alla popolazione urbana delle nazioni del Sud America nel 2019."

response = client.chat.completions.create(
    model="...",
    messages=[
        {"role": "system", "content": "Rispondi sempre in italiano." },
        {"role": "user", "content": prompt }
    ],
    max_tokens=-1,
    temperature=0.7,
    tools=tools # type: ignore
)

post_process_WB_API_call(exec_tool(response)) # type: ignore

[{'country': 'Brazil', 'population': 180121128},
 {'country': 'Argentina', 'population': 41371540},
 {'country': 'Chile', 'population': 16825479},
 {'country': 'Paraguay', 'population': 4031453},
 {'country': 'Venezuela, RB', 'population': 25534978},
 {'country': 'Ecuador', 'population': 11095186},
 {'country': 'Bolivia', 'population': 8143476}]